In [88]:
#Подключим нужные библиотеки
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score

from scipy.stats import randint, uniform

Так как ранее данные уже были проанализированны и очишенны от выбросов и неэффективгых признаков сразу приступаем к построению моделей.

In [101]:
#Загрузи данные
CC50_df = pd.read_csv('./data/CC50.csv')
display(CC50_df)

,Unnamed: 0,"CC50, mM",MaxEStateIndex,MinAbsEStateIndex,qed,SPS,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,...,fr_oxime,fr_piperzine,fr_priamide,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_thiazole,fr_thiophene,fr_urea
0,0,175.482382,5.094096,0.387225,0.417362,42.928571,-0.293526,0.293526,0.038844,0.642857,...,0,0,0,0,0,0,0,0,0,0
1,1,5.402819,3.961417,0.533868,0.462473,45.214286,-0.313407,0.313407,0.012887,0.607143,...,0,0,0,0,0,0,0,0,0,0
2,2,161.142320,2.627117,0.543231,0.260923,42.187500,-0.325573,0.325573,0.094802,0.562500,...,0,0,0,0,2,0,0,0,0,0
3,3,107.855654,5.097360,0.390603,0.377846,41.862069,-0.293526,0.293526,0.038844,0.620690,...,0,0,0,0,0,0,0,0,0,0
4,4,139.270991,5.150510,0.270476,0.429038,36.514286,-0.257239,0.257239,0.062897,0.600000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,996,34.999650,12.934891,0.048029,0.382752,49.133333,-0.468587,0.468587,0.317890,1.133333,...,0,0,0,0,0,0,0,0,0,0
886,997,33.999415,13.635345,0.030329,0.369425,44.542857,-0.467493,0.467493,0.327562,1.085714,...,0,0,0,0,0,0,0,0,0,0
887,998,33.999458,13.991690,0.026535,0.284923,41.973684,-0.467485,0.467485,0.327887,1.157895,...,0,0,0,0,0,1,0,0,0,0
888,999,32.999644,13.830180,0.146522,0.381559,39.000000,-0.468755,0.468755,0.312509,0.756757,...,0,0,0,0,0,0,0,0,0,0


Преобразим целеавой признак в True и False для выполнения классификации.

Значение True установим если значение CC50 > медианного значения иначе установим False.

Так как медианное значение это значение центрального элемента выборки, то имеем сбалансированный датасет для обучения.

In [102]:
#Преобразим целевой признак
target = CC50_df['CC50, mM'] >= CC50_df['CC50, mM'].median()

In [103]:
#Выведем признаки
features = CC50_df.drop(['CC50, mM', 'Unnamed: 0'], axis=1)

In [92]:
#Выполним стандартизацию данных
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [93]:
#Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

<h2>Модель логистической регрессии</h2>

In [94]:
model = LogisticRegression(
    max_iter=200
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Логистическая регрессия:')
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Логистическая регрессия:
Accuracy: 0.7752808988764045
              precision    recall  f1-score   support

       False       0.83      0.70      0.76        91
        True       0.73      0.85      0.79        87

    accuracy                           0.78       178
   macro avg       0.78      0.78      0.77       178
weighted avg       0.78      0.78      0.77       178



<h2>Модель DecisionTree</h2>

In [100]:
model = DecisionTreeClassifier(
    max_depth=7,
    min_samples_split=5,
    min_samples_leaf=4,
    random_state=42
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Дерево решений:')
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Дерево решений:
Accuracy: 0.702247191011236
              precision    recall  f1-score   support

       False       0.79      0.57      0.66        91
        True       0.65      0.84      0.73        87

    accuracy                           0.70       178
   macro avg       0.72      0.71      0.70       178
weighted avg       0.72      0.70      0.70       178



<h2>Модель Random Forest</h2>

In [96]:
#Обучение модели
model = RandomForestClassifier(
    n_estimators=72,
    max_depth=7,
    random_state=42
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Ансамбль деревьев:')
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Ансамбль деревьев:
Accuracy: 0.7921348314606742
              precision    recall  f1-score   support

       False       0.84      0.74      0.78        91
        True       0.76      0.85      0.80        87

    accuracy                           0.79       178
   macro avg       0.80      0.79      0.79       178
weighted avg       0.80      0.79      0.79       178



<h2>Модель SVС</h2>

In [97]:
# Обучение модели
model = SVC(
    kernel='rbf',
    class_weight='balanced',
    C=200
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Классивикация по опорным векторам:')
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Классивикация по опорным векторам:
Accuracy: 0.7584269662921348
              precision    recall  f1-score   support

       False       0.78      0.74      0.76        91
        True       0.74      0.78      0.76        87

    accuracy                           0.76       178
   macro avg       0.76      0.76      0.76       178
weighted avg       0.76      0.76      0.76       178



<h2>Модель KNN</h2>

In [98]:
# Обучение модели
model = KNeighborsClassifier(
    n_neighbors=4,
    weights='distance'
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Регрессия по K-ближайшим соседям:')
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Регрессия по K-ближайшим соседям:
Accuracy: 0.7640449438202247
              precision    recall  f1-score   support

       False       0.84      0.67      0.74        91
        True       0.71      0.86      0.78        87

    accuracy                           0.76       178
   macro avg       0.77      0.77      0.76       178
weighted avg       0.78      0.76      0.76       178



In [99]:
model = catboost = CatBoostClassifier(
    iterations=100,
    verbose=0,
    random_state=42
)

# Обучение модели
model.fit(X_train, y_train, eval_set=(X_test, y_test))

# Предсказание
y_pred = model.predict(X_test)
print('Регрессия CatBoost:')
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Регрессия CatBoost:
Accuracy: 0.7865168539325843
              precision    recall  f1-score   support

       False       0.84      0.71      0.77        91
        True       0.74      0.86      0.80        87

    accuracy                           0.79       178
   macro avg       0.79      0.79      0.79       178
weighted avg       0.79      0.79      0.79       178



<b>Для лучшей модели попробуем автоматический подбор гипер параметров</b>

In [11]:
param_dist = {
    'n_estimators': randint(3, 100),
    'max_depth': randint(2, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None]
}
random_search = RandomizedSearchCV(
    estimator= RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=500,
    scoring='roc_auc',
    cv=5,
    random_state=42,
    n_jobs=-10
)

random_search.fit(X_train, y_train)

best_params = random_search.best_params_
print("Лучшие параметры:", best_params)

best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

print('Лучшие метрики Random Forest при подборе:')
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))


Лучшие параметры: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}
Лучшие метрики Random Forest при подборе:
Accuracy: 0.7640449438202247
              precision    recall  f1-score   support

       False       0.82      0.69      0.75        91
        True       0.72      0.84      0.78        87

    accuracy                           0.76       178
   macro avg       0.77      0.77      0.76       178
weighted avg       0.77      0.76      0.76       178



<h2>Выводы</h2>

Для кластеризации препаратов по медианному значению <b>CC50</b>, были использованы основные модели классического ML, такие как:

* LogisticRegression;
* DecisionTree;
* Random Forest;
* SVС;
* KNN.

А так же готовое решение CatBoostClassifier.

Наилучшую метрику Accuracy показала модель Random Forest со значением 0.79, что считается не плохим результатом. При этом метрика Recall составляет 0.74 и 0.85 для препаратов со значением ниже и выше медианного IC50 соответственно. Стоит отметить, что модель предсказывает лучше при IC50 больше медианного.

Для подбора наилучших гипер параметров был использован алгоритм случайного поиска. Но ручной подбор гипер параметров оказался лучше.